<a href="https://colab.research.google.com/github/tvelichkovt/TensorFlow/blob/master/4_TensorFlow_%3E_tf_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



Test the Environment and TensorFlow ver > 2.x

---




In [117]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print('TensorFlow ver is:', tf.__version__, '\nKeras ver is:', keras.__version__, '\nPandas ver is:', pd.__version__, '\nNumpy ver is:', np.__version__)

TensorFlow ver is: 2.1.0 
Keras ver is: 2.2.4-tf 
Pandas ver is: 0.25.3 
Numpy ver is: 1.17.5




---



Download the csv file containing the heart dataset

---



In [118]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')
df = pd.read_csv(csv_file)
df.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [119]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [120]:
print('diagnosed ==', df.query("target == '1'")["target"].count())

diagnosed == 83


---


thal is object




---



In [121]:
df.dtypes #thal is object
df.thal = df.thal.astype('category').cat.codes

for i in df.columns:
        df[i]=df[i].astype(float)
df.dtypes

age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
target      float64
dtype: object

---
Convert thal column which is an object in the dataframe to a discrete numerical value.

---



In [0]:
#df['thal'] = pd.Categorical(df['thal'])
#df['thal'] = df.thal.cat.codes

---
tf.data.Dataset.from_tensor_slices

---



In [123]:
target = df.pop('target')
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0.0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1.0
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0.0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0.0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0.0


---
Since a pd.Series implements the __array__ protocol it can be used transparently nearly anywhere you would use a np.array or a tf.Tensor.

---



In [124]:
tf.constant(df['thal'])


<tf.Tensor: shape=(303,), dtype=float64, numpy=
array([2., 3., 4., 3., 3., 3., 3., 3., 4., 4., 2., 3., 2., 4., 4., 3., 4.,
       3., 3., 3., 3., 3., 3., 4., 4., 3., 3., 3., 3., 4., 3., 4., 3., 4.,
       3., 3., 4., 2., 4., 3., 4., 3., 4., 4., 2., 3., 3., 4., 3., 3., 4.,
       3., 3., 3., 4., 3., 3., 3., 3., 3., 3., 4., 4., 3., 3., 4., 4., 2.,
       3., 3., 4., 3., 4., 3., 3., 4., 4., 3., 3., 4., 4., 3., 3., 3., 3.,
       4., 4., 4., 3., 3., 4., 3., 4., 4., 3., 4., 3., 3., 3., 4., 3., 4.,
       4., 3., 3., 4., 4., 4., 4., 4., 3., 3., 3., 3., 4., 3., 4., 3., 4.,
       4., 3., 3., 2., 4., 4., 2., 3., 3., 4., 4., 3., 4., 3., 3., 4., 2.,
       4., 4., 3., 4., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4.,
       4., 4., 3., 3., 3., 4., 3., 4., 3., 4., 3., 3., 3., 3., 3., 3., 3.,
       4., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 3., 3.,
       3., 3., 3., 3., 3., 3., 4., 3., 4., 3., 2., 4., 4., 3., 3., 3., 3.,
       3., 3., 4., 3., 3., 3., 3., 3., 2., 2., 4., 3

---
Shuffle and batch the dataset.

---



In [0]:
train_dataset = dataset.shuffle(len(df)).batch(1)


---
Create and train a model

---



In [126]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model


model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Train for 303 steps
Epoch 1/15
303/303 [==============================] - 1s 3ms/step - loss: 0.6155 - accuracy: 0.7063
Epoch 2/15
303/303 [==============================] - 0s 2ms/step - loss: 0.5803 - accuracy: 0.7228
Epoch 3/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5591 - accuracy: 0.7360
Epoch 4/15
303/303 [==============================] - 0s 2ms/step - loss: 0.5362 - accuracy: 0.7327
Epoch 5/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7261
Epoch 6/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5288 - accuracy: 0.7558
Epoch 7/15
303/303 [==============================] - 0s 2ms/step - loss: 0.5432 - accuracy: 0.7327
Epoch 8/15
303/303 [==============================] - 0s 2ms/step - loss: 0.5260 - accuracy: 0.7327
Epoch 9/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5213 - accuracy: 0.7492
Epoch 10/15
303/303 [==============================] - 0s 2ms/step - loss: 0.513

---

Alternative to feature columns

---



In [0]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

---
The easiest way to preserve the column structure of a pd.DataFrame when used with tf.data is to convert the pd.DataFrame to a dict, and slice that dictionary.

---



In [128]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

for dict_slice in dict_slices.take(1):
  print (dict_slice)

model_func.fit(dict_slices, epochs=15)

({'age': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([63., 67., 67., 37., 41., 56., 62., 57., 63., 53., 57., 56., 56.,
       44., 52., 57.], dtype=float32)>, 'sex': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.],
      dtype=float32)>, 'cp': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([1., 4., 4., 3., 2., 2., 4., 4., 4., 4., 4., 2., 3., 2., 3., 3.],
      dtype=float32)>, 'trestbps': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([145., 160., 120., 130., 130., 120., 140., 120., 130., 140., 140.,
       140., 130., 120., 172., 150.], dtype=float32)>, 'chol': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([233., 286., 229., 250., 204., 236., 268., 354., 254., 203., 192.,
       294., 256., 263., 199., 168.], dtype=float32)>, 'fbs': <tf.Tensor: shape=(16,), dtype=float32, numpy=
array([1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0.],
      dtype=float32)>, 'restecg': <tf.T

---
[www](https://www.tensorflow.org/tutorials/load_data/pandas_dataframe)

---

